# Demonstration of Graphic facilities in gstlearn package

The module gstlearn.plot contains various plot functions for gstlearn objetcs: DbGrid, Db, Vario, Model, Polygons... These functions are also accesible as methods of each class. For example for a grid, we could use equivalently gp.grid(mygrid,...) or mygrid.plot(...), or for more specific functions: gp.point(mygrid,...) or mygrid.plot_point(...)

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

## Import packages

In [ ]:
import numpy as np
import pandas as pd
import sys
import os
import matplotlib
import matplotlib.pyplot as plt
import gstlearn as gl
import gstlearn.plot as gp
from scipy import ndimage, misc 

We define the Space dimension

In [ ]:
gl.defineDefaultSpace(gl.ESpaceType.RN, 2)

Creating a dummy Model used for simulating a random field and display it

In [ ]:
ctxt = gl.CovContext() # use default space
mymodel = gl.Model(ctxt)
cova = gl.CovAniso(gl.ECov.CUBIC,10,1,1.5,mymodel.getContext())
mymodel.addCov(cova)
ax = mymodel.plot(flagLegend=True)
ax.decoration(title="Cubic model")
plt.text(5,0.5,'$%s$'%cova.getFormula(),size='medium')
plt.show()

You can also directly display the formula in an output cell of a notebook

In [ ]:
from IPython.display import display, Latex
display(Latex('$%s$'%cova.getFormula()))

## Grid representations

We create a rectangular non-rotated 2-D grid, and simulate random Gaussian field (using the Model previously defined). Two simulations are generated in order to emphasize the graphic posibilities in further parts of this note.

In [ ]:
nx = [70,25]
dx = [1,2]
x0 = [-40, 20]
mygrid = gl.DbGrid.create(nx,dx,x0)

err = gl.simtub(None,mygrid,mymodel,None,2)
mygrid.display()

Add a dummy selection to test visualization with Selection

In [ ]:
mygrid["sel"] = 1. - (mygrid["x1"] > 0) * (mygrid["x1"] < 15) * (mygrid["x2"] > 40) * (mygrid["x2"] < 50)
mygrid.setLocator("sel",gl.ELoc.SEL)

### Non Rotated Grid

We simply represent the grid (using the defaulted color scale). As no variable nor representation is explicitely specified, the default variable is represented using the default representation (i.e. raster). The default variable is the first Z-locator variable (if any) or the variable created last in the file otherwise.

We also plot the legend on the right edge.  

In [ ]:
ax = gp.grid(mygrid, name_raster="Simu.1")
ax.decoration(title="Non-conditional Simulation on Grid")

This can be specified using the object-based syntax instead, as follows:

In [ ]:
ax = mygrid.plot(name_raster="Simu.1")
ax.decoration(title="Non-conditional Simulation on Grid")

We can overlay a set of isolines (based on the second simulation)

In [ ]:
ax = mygrid.plot(name_raster="Simu.1",name_contour="Simu.2",levels=[-2,0,1,2,3,4,5,6,7,8]) 

As a proof, we can also visualize the (last) variable using a Color Point representation. Note that here we **cannot** use the object-based convention as we wish to use the *point* representation facility, even if the file is organized as a grid.  

In [ ]:
ax = gp.point(mygrid,name_color=mygrid.getLastName()) # last name is the selection

We can create a specific ColorScale, containing a limited number of colors, and sampling a given reference Color Scale. 

For the next figure, we use the one defaulted by the system ('viridis') for sake of understanding. We simply reduce the number of colors

In [ ]:
cmap = gp.get_cmap(5,'viridis')
ax = mygrid.plot(cmap=cmap)
ax.decoration(title="Non-conditional Simulation on Grid")

We can also change the reference color scale (using the one defaulted by the method get_cmap for example: 'gist rainbow') and increase the number of colors

In [ ]:
cmap = gp.get_cmap(100)
ax = mygrid.plot(cmap=cmap)
ax.decoration(title="Non-conditional Simulation on Grid")

Represent the histogram of the vaalues collected in the outcome over the grid. Here we can use either gp.hist(mygrid,...) or mygrid.plot_hist(...)

In [ ]:
ax = mygrid.plot_hist("Simu.1", bins=30, color='yellow', edgecolor="blue")
ax.decoration(xlabel="Simulated Values",ylabel="Count",title="Histogram")

Representing a scatter plot between two variables stored on the same Db

In [ ]:
ax = mygrid.plot_correlation("Simu.1","Simu.2", bins=50, cmin=1)

### Set of Points and Polygon

A set of points is sampled from the previous Grid and stored in a new Point Db. The number of samples if fixed to 1% of the number of grid nodes.

In [ ]:
mypoint = gl.Db()
mypoint.resetSamplingDb(mygrid,0.01)
mypoint.display()

We create a polygon as the convex hull of the samples

In [ ]:
mypoly = gl.Polygons.createFromDb(mypoint)

We now display the points and the polygon on top of the grid: the overlay is ensured by using the argument 'ax'.

In [ ]:
ax = gp.grid(mygrid)
ax = gp.polygon(mypoly,flagFace=False, edgecolor='yellow', linewidth=2, ax=ax)
ax = gp.point(mypoint,color="black", ax=ax)

### Rotated grid (angle = 20 degrees)

We create the same grid as before but with a rotation of 20 degrees.

In [ ]:
mygrid = gl.DbGrid.create(nx,dx,x0,[20,0])
err = gl.simtub(None,mygrid,mymodel,nbsimu=2)
ax = mygrid.plot(name_raster="Simu.1")

A new set of Points is sampled from the rotated Grid. As the same seed is used, the ranks of the selected samples within the grid are the same. Furthermore, we generate the Polygon as the convex hull of the newly created Point db.

In [ ]:
mypoint = gl.Db()
mypoint.resetSamplingDb(mygrid,0.01)
mypoly = gl.Polygons.createFromDb(mypoint)

We represent again the three components (grid, points and polygon) on the same view

In [ ]:
ax = mygrid.plot()
ax = mypoly.plot(flagFace=False,edgecolor='yellow',ax=ax)
ax = mypoint.plot(color="black",ax=ax)

Let us now add a selection in order to restrict the previous representation to the only non-masked samples

In [ ]:
tab = mygrid.getColumn("x1")
sel = (np.asarray(tab) < 0).astype(float)
mygrid.addSelection(tuple(sel),'sel')
ax = gp.grid(mygrid,name_raster="Simu.1",usesel=True)

## Share legend between two plots
This paragraph is meant to present the possibility of splitting a figure in two scenes, to represent a grid in each scene (for example) and share the (same) color scale for the two scenes.

In [ ]:
fig = plt.figure(figsize=(20,6))
ax1 = fig.add_subplot(1,2,1)
gp.grid(mygrid,name_raster="Simu.1", usesel=False, flagLegendRaster = False, ax=ax1)
ax2 = fig.add_subplot(1,2,2)
gp.grid(mygrid,name_raster="Simu.2", usesel=False, flagLegendRaster = False, ax=ax2)

fig.subplots_adjust(right=0.7)
cbar_ax = fig.add_axes([0.75, 0.1, 0.02, 0.75])

im = ax.collections[0] # get mappable described by the colorbar
err = fig.colorbar(im, cax = cbar_ax)

## Display points with fixed colors

In this paragraph, we wish to display sample points with given colors. 

In [ ]:
tab = [1., 1., 1., 2., 2., 3., 3., 3., 5.]
dat1 = gl.Db.createFromSamples(3, gl.ELoadBy.SAMPLE, tab, ["x","y","z"], ["x1","x2","z"])
dbfmt = gl.DbStringFormat()
dbfmt.setFlags(flag_resume=True, flag_array=True)
dat1.display(dbfmt)

We represent the samples without using any pre-specified color map. The system uses the default color scale and assigns the lowest value to be represented to the first color and the largest value to the last color.
For all subsequent graphics, the dimensions of the non-geographical plots is fixed.

In [ ]:
gp.setDefaultGeographic(dims=[5,4])
ax = gp.point(dat1,name_color="z",size=200)

Representing using a given color map (based on few colors [5]). The color scale is now discrete but the system still assigns the lowest value (i.e. 1) to the first color and the largest value (i.e. 5) to the last color

In [ ]:
ncol = 5
cmap = gp.get_cmap(ncol)

In [ ]:
ax = gp.point(dat1,name_color="z",size=200,cmap=cmap)

We use a new Db where the values at first sample has been modified (from 1 to 4) while the other have been left unchanged. We use the same color scale as before. Again the lowest value (i.e. 3) is assigned to the first color and the largest value (i.e. 5) to the last color.

In [ ]:
tab = [1., 1., 4., 2., 2., 3., 3., 3., 5.]
dat2 = gl.Db.createFromSamples(3, gl.ELoadBy.SAMPLE, tab, ["x","y","z"], ["x1","x2","z"])
ax = gp.point(dat2,name_color="z",size=200,cmap=cmap)

Default dimensions for Geographical plots are set back to larger dimensions for subseauqnt graphics

In [ ]:
gp.setDefaultGeographic(dims=[8,8])

# Superimposing figures

In this section, we demonstrate the possibilities offered by the graphics for working with multiple figures and overlaying graphics. This is described through the use of variograms and models.
For this reason we consider the two non-conditional simulations created earllier on the existing grid. We calculate the simple and cross variograms along the two main axes of the grid and fit a model automatically.

In [ ]:
varioparam = gl.VarioParam.createMultipleFromGrid(npas=10)
vario = gl.Vario(varioparam, mygrid)
err = vario.compute(gl.ECalcVario.VARIOGRAM)

In [ ]:
model = gl.Model()
err = model.fit(vario,[gl.ECov.CUBIC])

### Several ways for displaying experimental variograms

In the next graphic, we produce the simple variogram of the first variable calculated in the first direction

In [ ]:
ax = vario.plot()
gp.geometry(ax, dims=[6,5])
gp.decoration(ax, title="First Variable - First Direction")

In the next graphic, we produce a single figure where the variograms of the first variable calculated in the first direction (black) and the second direction (red) are overlaid. The overlay is performed manually.

In [ ]:
ax = vario.plot(color="black")
ax = vario.plot(idir=1, color='red', axs=ax)
ax.decoration(title="First Variable - Two directions")
ax.geometry(dims=[6,5])

In the next graphic, we produce a single graphic where the cross-variograms between first and second variables are displayed for all directions. The colors are extracted from the Color Map provided as argument.

In [ ]:
ax = vario.plot(ivar=1,jvar=0,idir=-1)
ax.decoration(title="Cross-variogram - Two directions")
ax.geometry(dims=[6,5])

Representing all simple and cross variograms for all directions

In [ ]:
axs = vario.plot(ivar=-1,jvar=-1,idir=-1,cmap=cmap)
gp.decoration(axs,title="Simple and Cross Variograms in all directions")
gp.geometry(axs, dims=[6,5])

### Several ways for representing the Model

Represent the Model calculated for the second variable. If the Model is not isotropic, the plot should differ per direction: as direction has not been mentionned, the first direction (of the geographic sysrtem) is used by default.

In [ ]:
ax = gp.model(model,ivar=1,jvar=1)
ax.decoration(title="Model for the Second Variable in First Direction")
ax.geometry(dims=[6,5])

Representing all simple and cross variograms together with the fitted model for the first direction. This is directly provided by the function varmod.

In [ ]:
axs = gp.varmod(vario, model, cmap=cmap)
gp.decoration(axs, title="All variograms in First Direction")
gp.geometry(axs, dims=[6,5])

### Testing figure Overlay

The next figure is meant to demonstrate the overlay possibilities. We first represent the experimental variograms for all variable (in the first direction only to be legible). Then we overlay the model ... only over the experimental simple variogram of the second variable (in dashed blue).

In [ ]:
axs = vario.plot(idir=0,ivar=-1,jvar=-1,cmap=cmap)
gp.decoration(axs, title="Overlay test")
gp.geometry(axs, dims=[6,5])
ax = model.plot(ivar=1,jvar=1,codir=vario.getCodirs(0),hmax = vario.getHmax(),
                linestyle = 'dashed', color='blue',ax=axs[1,1])

In the next figure, we draw the first direction and overlay the second direction (on purpose using two orders)

In [ ]:
axs = vario.plot(idir=0,cmap=cmap)
axs = vario.plot(idir=1,axs=axs)
gp.decoration(axs, title="Overlay test")
gp.geometry(axs, dims=[4,3])